In [1]:
using LinearAlgebra
P = [ 1.0  0.0  1.0  2.0
     -1.0  0.0  1.0  2.0
      0.0  1.0  0.0 -3.0
      0.0 -1.0  0.0 -3.0
      0.0  0.0 -2.0  2.0]
for i in 1:4
    P[:, i] /= norm(P[:, i])
end
@show P' * ones(5)
round.(P' * P, digits=16)

P' * ones(5) = [0.0, 0.0, 0.0, 0.0]


4×4 Array{Float64,2}:
 1.0   0.0   0.0   0.0
 0.0   1.0   0.0  -0.0
 0.0   0.0   1.0  -0.0
 0.0  -0.0  -0.0   1.0

In [2]:
using HybridSystems
H1 = [0.2 0.8 0   0   0
      0   0.2 0.8 0   0
      0   0   0.2 0.8 0
      0   0   0   0.2 0.8
      0.2 0   0   0   0.8]
A1 = P' * H1 * P
H2 = [1   0   0   0   0
      0.8 0.2 0   0   0
      0   0   1   0   0
      0   0   0.8 0.2 0
      0.2 0   0   0   1]
A2 = P' * H2 * P
H3 = [1   0   0   0   0
      0   1   0   0   0
      0   0.8 0.2 0   0
      0   0   0   1   0
      0   0   0   0.2 0.8]
A3 = P' * H3 * P
function automaton(N)
    # See [P17, Figure 2.17] for what automaton(3) should be
    a = LightAutomaton(N)
    # Node i means, H1 was used i-1 steps ago
    add_transition!(a, 1, 1, 1)
    for i in 2:N
        add_transition!(a, i-1, i, 2)
        add_transition!(a, i-1, i, 3)
        add_transition!(a, i, 1, 1)
    end
    display(a)
    return a
end
ss(N) = discreteswitchedsystem([A1, A2, A3], automaton(N))

ss (generic function with 1 method)

In [3]:
for t in transitions(automaton(2))
    @show t
end

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({2, 3} directed simple Int64 graph, Dict(Edge 1 => 2=>Dict(2=>2,3=>3),Edge 2 => 1=>Dict(4=>1),Edge 1 => 1=>Dict(1=>1)), 4, 4)

t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 1, 1)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 2, 2)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 2, 3)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 2 => 1, 4)


In [4]:
using SwitchOnSafety
ρ(A1)

┌ Info: Precompiling SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510]
└ @ Base loading.jl:1189
ERROR: LoadError: LoadError: TypeError: in Type{...} expression, expected UnionAll, got TypeVar
Stacktrace:
 [1] top-level scope at none:0
 [2] include at ./boot.jl:317 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1041
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /home/blegat/.julia/dev/SwitchOnSafety/src/SwitchOnSafety.jl:1
 [6] top-level scope at none:0
 [7] top-level scope at none:2
in expression starting at /home/blegat/.julia/dev/SwitchOnSafety/src/sosdata.jl:3
in expression starting at /home/blegat/.julia/dev/SwitchOnSafety/src/SwitchOnSafety.jl:71


ErrorException: Failed to precompile SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510] to /home/blegat/.julia/compiled/v1.0/SwitchOnSafety/PA0qe.ji.

In [5]:
using MathOptInterfaceMosek
using JuMP
factory = with_optimizer(MosekOptimizer, QUIET=true);

In [6]:
using SwitchOnSafety
soslyapb(ss(1), 1; factory=factory, tol=1e-7, verbose=1)

┌ Info: Precompiling SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510]
└ @ Base loading.jl:1189
ERROR: LoadError: LoadError: TypeError: in Type{...} expression, expected UnionAll, got TypeVar
Stacktrace:
 [1] top-level scope at none:0
 [2] include at ./boot.jl:317 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1041
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /home/blegat/.julia/dev/SwitchOnSafety/src/SwitchOnSafety.jl:1
 [6] top-level scope at none:0
 [7] top-level scope at none:2
in expression starting at /home/blegat/.julia/dev/SwitchOnSafety/src/sosdata.jl:3
in expression starting at /home/blegat/.julia/dev/SwitchOnSafety/src/SwitchOnSafety.jl:71


ErrorException: Failed to precompile SwitchOnSafety [4b7ae085-addf-5b2e-b0f2-71171a701510] to /home/blegat/.julia/compiled/v1.0/SwitchOnSafety/PA0qe.ji.

In [45]:
using SwitchOnSafety
soslyapb(ss(2), 1; factory=factory, tol=1e-4, verbose=1)

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({2, 3} directed simple Int64 graph, Dict(Edge 1 => 2=>3,Edge 2 => 1=>1,Edge 1 => 1=>1))

Lower bound: 0.9082094659721799
Upper bound: 0.9082649003705496
Log-diff   : 6.103515625002776e-5 ≤ 0.0001


(0.7139901229663629, 0.9082649003705496)

In [42]:
using SwitchOnSafety
soslyapb(ss(3), 1; factory=factory, tol=1e-4, verbose=1)

Lower bound: 0.9652509577084115
Upper bound: 0.9653098737493923
Log-diff   : 6.103515624994449e-5 ≤ 0.0001


(0.7117304054040449, 0.9653098737493923)

In [43]:
using SwitchOnSafety
soslyapb(ss(4), 1; factory=factory, tol=1e-4, verbose=1)

Lower bound: 0.981169815633469
Upper bound: 0.9812297033140823
Log-diff   : 6.103515625002776e-5 ≤ 0.0001


(0.7067079941582048, 0.9812297033140823)